<a href="https://colab.research.google.com/github/mani843/AI_Bootcamp/blob/main/Imran_Speech_Emotion_Recognition_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
Root = "/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)


In [3]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/  modelForPrediction1.sav
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  Actor_22/  our_model.sav
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/  Actor_23/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/  Actor_24/


In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [5]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [6]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [7]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [8]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [9]:
x_train

array([[-7.07727539e+02,  6.59930954e+01,  6.94958067e+00, ...,
         2.84301386e-05,  1.72711170e-05,  7.26592498e-06],
       [-4.55947113e+02,  1.13258772e+01, -1.81083488e+01, ...,
         1.40275690e-03,  7.50296458e-04,  4.46032209e-04],
       [-6.47927795e+02,  4.18512344e+01, -6.38385439e+00, ...,
         9.40296741e-05,  6.72010283e-05,  4.72025567e-05],
       ...,
       [-4.53225311e+02,  3.05153828e+01, -2.49701138e+01, ...,
         6.04014320e-04,  3.06453265e-04,  1.45317535e-04],
       [-4.72468048e+02,  5.39467812e+00, -3.28353958e+01, ...,
         5.81849483e-04,  5.11193881e-04,  3.23587621e-04],
       [-6.36445862e+02,  4.03178978e+01, -2.45225735e+01, ...,
         9.02314969e-06,  4.44445732e-06,  7.63565095e-06]])

In [10]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [11]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [12]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [13]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [14]:
#Predict for the test set
y_pred=model.predict(x_test)

In [15]:
y_pred

array(['disgust', 'disgust', 'disgust', 'fearful', 'disgust', 'calm',
       'happy', 'fearful', 'disgust', 'calm', 'happy', 'disgust',
       'disgust', 'fearful', 'disgust', 'disgust', 'happy', 'disgust',
       'disgust', 'disgust', 'disgust', 'disgust', 'calm', 'calm', 'calm',
       'fearful', 'disgust', 'disgust', 'fearful', 'disgust', 'happy',
       'disgust', 'happy', 'fearful', 'happy', 'disgust', 'disgust',
       'fearful', 'happy', 'calm', 'happy', 'fearful', 'disgust', 'calm',
       'fearful', 'calm', 'disgust', 'fearful', 'calm', 'fearful',
       'happy', 'happy', 'disgust', 'fearful', 'calm', 'disgust',
       'fearful', 'happy', 'calm', 'fearful', 'fearful', 'disgust',
       'happy', 'calm', 'disgust', 'calm', 'calm', 'fearful', 'disgust',
       'calm', 'fearful', 'happy', 'disgust', 'fearful', 'fearful',
       'fearful', 'disgust', 'disgust', 'disgust', 'fearful', 'disgust',
       'disgust', 'disgust', 'happy', 'disgust', 'fearful', 'disgust',
       'calm', 'di

In [16]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 66.15%


In [17]:
from sklearn.metrics import accuracy_score, f1_score

In [18]:
f1_score(y_test, y_pred,average=None)

array([0.75294118, 0.60162602, 0.68181818, 0.63636364])

In [19]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,disgust
1,calm,disgust
2,happy,disgust
3,happy,fearful
4,disgust,disgust
5,calm,calm
6,happy,happy
7,happy,fearful
8,disgust,disgust
9,happy,calm


In [20]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [21]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [22]:
feature

array([[-6.93497009e+02,  5.00643883e+01,  5.71450472e-01,
         1.43299656e+01,  3.33637023e+00, -2.54071975e+00,
        -4.05790901e+00, -1.07119989e+01, -7.29413891e+00,
         1.74018884e+00, -4.19064283e+00,  1.95466173e+00,
        -5.24789333e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803221e+00, -5.68717480e-01,
        -6.47753334e+00, -1.24320579e+00, -2.80542541e+00,
        -5.43635845e+00, -4.46875453e-01, -3.63516617e+00,
        -2.98372650e+00, -5.63902617e-01, -1.65101981e+00,
        -5.55944800e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821268e+00,
        -5.54154396e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577050e+00, -1.62457860e+00, -2.03990722e+00,
        -3.62910819e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345674e-01,
         6.60464048e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921480e-01,  7.23420441e-01,  7.16119647e-0

In [23]:
import joblib

In [24]:
# save the model to disk
drive_path = '/content/drive/MyDrive/Colab_Notebooks/'

filename = 'SER_model.sav'
joblib.dump(model, drive_path + filename)

['/content/drive/MyDrive/Colab_Notebooks/SER_model.sav']

In [25]:
loaded_model = joblib.load('/content/drive/MyDrive/Colab_Notebooks/SER_model.sav')

In [26]:
result = loaded_model.score(x_test, y_test)
print(result)

0.6614583333333334


In [27]:
!pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━

In [28]:
import gradio as gr
import joblib
our_trained_model =joblib.load("/content/drive/MyDrive/Colab_Notebooks/SER_model.sav")

In [30]:
def SER(file):
  feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
  feature=feature.reshape(1,-1)
  prediction=our_trained_model.predict(feature)
  return prediction

In [43]:
input_audio = gr.Audio(
    sources=["microphone"],
    streaming=False,
    type="numpy",
    label="Input Audio",
)

In [44]:
demo = gr.Interface(
    fn=SER,
    inputs=input_audio,
    outputs=['text'],
)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a4ea1e4baffd400342.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [45]:
prediction

array(['fearful'], dtype='<U7')